# 查询（query）和映射（projection）操作符

## 查询选择器（query selectors）
### 比较型的（comparison）
#### $eq
```
db.inventory.find({qty:{$eq:20}})
# 等同于
db.inventory.find({qty:20})
# 当有嵌套文档时，注意引号
db.inventory.find({"item.name":{$eq:"ab"}})
# 匹配数组
    # 此时的返回结果对应的数组或完全等于该值或者包含该值
# 匹配正则表达式
```

#### \\$gte \\$gt  \\$lt \\$lte  \\ne

#### \\$in
```
db.inventory.find( { qty: { $in: [ 5, 15 ] } } )
# 数组, tags对应的值时数组,其中有元素是"abc"或"school"
{ _id: 1, item: "abc", qty: 10, tags: [ "school", "clothing" ], sale: false }
db.inventory.update(
    { tags: { $in: ["abc", "school"] } },
    { $set: { scale: true } }
)
# 正则表达式
db.inventory.find( { tags: { $in: [ /^be/, /^st/ ] } } )
# 以上查找的文档中的tags，如果是string格式，则其或者时以be开头的或者时以st开头的;如果是数组，则数组中的元素至少有一个是以be开头或者st开头
```
#### \\$nin 
\\$nin是\\$in反命题

### 逻辑型
#### \\$and
```
# 查询price字段存在且不等于1.99
db.inventory.find({$and:[{$price:{$ne:1.99}},{price,{$exists: true}}]})
# 等价与
db.inventory.inf({price:{$ne:1.99, $exists:true}})
# 查询（qty小于10或者大于50）且（sale是true或者price小于5）
db.inventory.find
({
    $and:[
          {$or:[{qty:{$lt:10}}, {qty:{$gt:50}}],
          {$or:[{sale:true},{price:{$lt:5}}]}
         ]
})          
```

#### \\$not
```
# 查找price小于等于1.99或者price字段不存在的文档
db.inventory.find({price:{$not:{$gt:1.99}}})
# 使用正则表达式对象查找itme字段不是以p字母开头
db.inventory.find({itme:{$not: /^p.*/}})
#使用$regex算子
db.inventory.find({item:{$not:{$regx:"^p.*"}}})
db.inventory.find({item:{$not:{$regx:/^p.*/}}})
```
**python中的正则表达式使用**

```
import re
for noMatch in db.inventory.find({"item":{$not:re.compile("^p.*")}}):
    print(noMatch)
```    

#### $nor
其中所有条件都不满足才行
```
# 查找price不等于1.99的或者price不存在的;或者sale存在且为false的或者sale不存在的
db.inventory.find({$nor:[{price:1.99},{sale:true}]})
# 查找price和sale都存在且price不等于1.99且sale不为true的
db.inventory.find({$nor:[{price:1.99},{price:{$exists:false}},{sale:true},{sale:{$exists:false}}]})
```

#### \\$or
```
# quantity小于20或price为10
db.inventory.find( { $or: [ { quantity: { $lt: 20 } }, { price: 10 } ] } )
```
*or里面每一个子查询（clause）都需要有索引，这样在进行or时会使用索引扫描（index scan），否则会进行集合扫描(collection scan)*

### 元素型（element）

#### \\$exists
```
# 查找qty字段存在，且不为5或15的文档
db.inventory.find({qty:{$exists:true,$nin:[5,15]}})
# 查找b字段不存在的文档
db.records.find( { b: { $exists: false } } )
```
#### \\$type
```
# 查询zipCode字段为string类型，可以是数组，数组里面是string
# 2是数字表示，string是代号（alias）表示的BSON数据类型
db.addressBook.find( { "zipCode" : { $type : 2 } } );
db.addressBook.find( { "zipCode" : { $type : "string" } } );
# 或是string或是double**注意类型得用双引号**
db.grades.find({"classAverage": {$type:[2, 1]}})
db.grades.find({"classAverage": {$type:["string", "double"]}})
# 查询readings字段是数组，空数据也算
db.SensorReading.find( { "readings" : { $type: "array" } } )
```

### Evaluation（计算型？）

#### \\$expr
```
# 查找spent的值大于budget的值的文档，**注意符号$**
db.monthlyBudget.find({ $expr: { $gt: [ "$spent", "$budget"]}})
```

```
# 插入所需文档
db.supplies.insertMany([
   { "_id" : 1, "item" : "binder", "qty" : NumberInt("100"), "price" : NumberDecimal("12") },
   { "_id" : 2, "item" : "notebook", "qty" : NumberInt("200"), "price" : NumberDecimal("8") },
   { "_id" : 3, "item" : "pencil", "qty" : NumberInt("50"), "price" : NumberDecimal("6") },
   { "_id" : 4, "item" : "eraser", "qty" : NumberInt("150"), "price" : NumberDecimal("3") },
   { "_id" : 5, "item" : "legal pad", "qty" : NumberInt("42"), "price" : NumberDecimal("10") }
])
# 计算discountedPrice
let discountedPrice = {
   $cond: {
      if: { $gte: ["$qty", 100] },
      then: { $multiply: ["$price", NumberDecimal("0.50")] },
      else: { $multiply: ["$price", NumberDecimal("0.75")] }
   }
};
# 查询discountedPrice小于5的
db.supplies.find( { $expr: { $lt:[ discountedPrice,  NumberDecimal("5") ] } });
```

#### $jsonSchema
**用于文档验证，该集合插入的文档需要满足以下要求才行，否则会插入失败（默认），也可以插入成功，但会提醒。**
```
db.createCollection("students", {
   validator: {
      $jsonSchema: {
         bsonType: "object",
         required: [ "name", "year", "major", "address" ],
         properties: {
            name: {
               bsonType: "string",
               description: "must be a string and is required"
            },
            year: {
               bsonType: "int",
               minimum: 2017,
               maximum: 3017,
               description: "must be an integer in [ 2017, 3017 ] and is required"
            },
            major: {
               enum: [ "Math", "English", "Computer Science", "History", null ],
               description: "can only be one of the enum values and is required"
            },
            gpa: {
               bsonType: [ "double" ],
               description: "must be a double if the field exists"
            },
            address: {
               bsonType: "object",
               required: [ "city" ],
               properties: {
                  street: {
                     bsonType: "string",
                     description: "must be a string if the field exists"
                  },
                  city: {
                     bsonType: "string",
                     "description": "must be a string and is required"
                  }
               }
            }
         }
      }
   }
})
```

**条件查询**
```
let myschema = {} # 写好的一个jsonschema
# 使用该jsonschema进行查询
db.inventory.find({$jsonSchema: myschema})
db.inventory.aggregate([{$match:{$jsonSchema: myschema}}])
# 查找不符合该jsonshema的然后设置新的字段isvalid为false
db.inventory.updateMany({$ nor:[{$jsonSchema:myschema}]},{$set:{ isvalid:false}})
```

#### $mod
```
# 查找qty的值除以4之后余0的文档
db.inventory.find({qty:{$mod:[4,0]}})
```

#### $regex
```
# 两种形式,某些情况下只能用其中的一种
{<field>: /pattern/<options>}
{<field>: {$regex: /pattern/<options>}}
{ <field>: { $regex: /pattern/, $options: '<options>' } }
{ <field>: { $regex: 'pattern', $options: '<options>' } }
# 在$in里不能使用$regex
{name:{$in:[/^acme/i, /^ack/]}} 
# 查找item不是以p开头的
db.inventory.find({item:{$not:{$regex:"^p.*}}})
# 查找name a不区分大小写，cme区分大小写，（?i）的作用域为剩余部分
{name :{ $regex: '(?i)a(?-i)cme'}}
```

#### $text
```
# 使用前需要由text的index
db.articles.createIndex( { subject: "text" } )
# 查找还有coffee单词的,不分大小写和音调符号
db.articles.find({$text: {$search :"coffee"}})
# 查找包含bake或者coffe或者cake的,有一个就行
db.articles.find({$text:{$search:"coffee bake cake"}})
# 查找包含“coffee shop”整个短语的
db.aritiles.find({$text:{$search:"\"coffee shop\""}})
# 查找由coffee但是后面不是跟着shop的
db.articles.find({$text:{$search:"coffee -shop"}})
# 查找包换Coffee的，区分大小写
db.articles.find({$text:{$seach:"Coffee",$caseSensitive:true}})
# $diacriticSensitive:true用于区分音调符号
```